# Sheet 10

## a.

**Tree after up-sweep**

In [47]:
temp = a
uptree = [a]
while len(temp) != 1:
    temp = [temp[b] + temp[b+1] for b in range(0, len(temp), 2)]
    uptree.append(temp)

In [48]:
print(uptree)

[[4, 6, 1, 0, 1, 7, 3, 2], [10, 1, 8, 5], [11, 13], [24]]


**Tree after down-sweep**

In [ ]:
temp = uptree[0:(len(uptree)-1)]
downtree = [[0]]
while len(temp) != 0:
    parents = downtree[-1]
    children = temp.pop()
    new_children = []
    for j in range(len(parents)):
        new_children.append(parents[j])
        new_children.append(children[2*j] + parents[j])
    downtree.append(new_children)

hi
hi
hi


In [104]:
print(downtree)

[[0], [0, 11], [0, 10, 11, 19], [0, 4, 10, 11, 11, 12, 19, 22]]


## b.
**What is the time complexity of the parallel scan algorithm? Explain why.**

If the inner loop over the trees nodes at a given depth is parallelized and has time complexity $O(1)$, then the time complexity of the parallel scan algorithm is determined by the depth of the tree. For every step up the tree the number of nodes is halved, resulting in the following relation between depth $D$ and number of nodes $N$:
$$N = 2^{D} \Leftrightarrow D = \frac{\log N}{\log 2}$$
Which means that the time complexity of the algorithm is $O(\log N)$. 

## c.
**Reformulate the recurrence in eq. II such that it fits eq. I.**

## d.

In [1]:
from jax.lax import associative_scan
from jax.random import randint, PRNGKey
import jax.numpy as np
import time
import os
from matplotlib import pyplot as plt

os.environ["CUDA_VISIBLE_DEVICES"] = "0" ### SET THIS TO THE GPU ID YOU WANT TO USE (leave at 0 if you don't know)
os.environ["JAX_PLATFORM_NAME"] = "cuda"

ModuleNotFoundError: No module named 'jax'

In [2]:
def cumsum_sequential(sequence):
    pass

def cumsum_associative_scan(sequence):
    def associative_binary_operator(element1, element2):
        pass
    pass

In [9]:
lengths = [10, 100, 1000, 10000]
times_autoregressive = []
times_associative_scan = []

for length in lengths:
    sequence = randint(PRNGKey(0), (length, ), 0, 10,)

    # your code here

In [ ]:
# visualize the results